In [20]:
import torch, clip, json
from baukit import pbar
torch.set_grad_enabled(False)

clip_model, clip_preprocess = clip.load('ViT-B/32')

In [11]:
with open('/data/datasets/coco/annotations/captions_train2017.json') as f:
    data = json.load(f)
    captions = [r['caption'] for r in data['annotations']]

In [23]:
result = []

for caption in pbar(captions[:100000]):
    tx = clip.tokenize(caption)
    out = clip_model.encode_text(tx.cuda()).cpu()
    result.append(out)

clip_text_encodings = torch.cat(result)

  0%|          | 0/100000 [00:00<?, ?it/s]

In [38]:
import os, numpy
dirname = 'clip_sample'
os.makedirs(dirname, exist_ok=True)
numpy.save(f'{dirname}/clip_text_encoding.npy', clip_text_encodings.numpy())

with open(f'{dirname}/clip_text.json', 'w') as f:
    json.dump(captions[:100000], f, indent=0)


In [29]:
from baukit import ParallelImageFolders

ds = ParallelImageFolders(['/data/datasets/coco/images/train2017'], transform=clip_preprocess)

AttributeError: 'tuple' object has no attribute 'shape'

In [37]:
loader = torch.utils.data.DataLoader(
    ds, batch_size=20, pin_memory=True, num_workers=8)
image_results = []
for [batch] in pbar(loader):
    image_encodings = clip_model.encode_image(batch.cuda())
    image_results.append(image_encodings.cpu())
clip_image_encodings = torch.cat(image_results)


  0%|          | 0/5915 [00:00<?, ?it/s]

NameError: name 'numpy' is not defined

In [39]:
import numpy
numpy.save(f'{dirname}/clip_image_encoding.npy', clip_image_encodings.numpy())